In [1]:
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

from collections import Counter
import re
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#!curl -LJO https://www.gutenberg.org/ebooks/67724.txt.utf-8
#!curl -LJO https://www.gutenberg.org/ebooks/67725.txt.utf-8


In [3]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()
text = text.lower()
print(len(text.split()))

paragraphs = text.split("\n\n")

len(paragraphs)

115112


4971

In [4]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]
print(cleaned_paragraphs)
len(cleaned_paragraphs)

['\ufeffthe project gutenberg ebook of o guarany: romance brazileiro, vol. 1 (of 2)      this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever. you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org. if you are not located in the united states, you will have to check the laws of the country where you are located before using this ebook.', 'title: o guarany: romance brazileiro, vol. 1 (of 2)', 'author: josé martiniano de alencar', 'release date: march 28, 2022 [ebook #67724]', 'language: portuguese', 'original publication: brazil: b. l. garnier, 1883', 'credits: laura natal rodrigues and kristen carmean (images generously made available by hathi trust digital library.)', ' *** start of the project gutenberg ebook o guarany: romance brazileiro, vol. 1 (of 2) ***', 'j. de alencar', ' o guarany', 'ro

4892

In [5]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+|[.,!?-]', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

print(len(word_counts))
print(word_counts)

12608
Counter({'.': 8870, ',': 7693, '-': 6223, 'a': 4595, 'que': 4340, 'o': 4079, 'de': 3960, 'e': 3658, 'se': 2402, 'um': 1711, 'do': 1442, 'não': 1280, 'uma': 1250, 'da': 1133, 'os': 1123, 'com': 1015, 'sua': 925, 'para': 857, 'seu': 777, '!': 773, 'pery': 732, 'as': 726, 'em': 724, 'no': 664, '?': 628, 'por': 622, 'ao': 594, 'como': 594, 'lhe': 558, 'd': 493, 'á': 490, 'tinha': 478, 'era': 469, 'cecilia': 457, 'na': 455, 'é': 441, 'sobre': 416, 'mas': 410, 'elle': 407, 'the': 376, 'dos': 373, 'indio': 340, 'me': 325, 'seus': 324, 'mais': 318, 'antonio': 303, 'quando': 288, 'alvaro': 278, 'disse': 259, 'das': 258, 'vos': 254, 'of': 252, 'ella': 233, 'olhos': 227, 'te': 227, 'senhora': 227, 'menina': 215, 'pela': 213, 'tu': 204, 'depois': 200, 'nos': 200, 'isabel': 197, 'havia': 195, 'gutenberg': 194, 'fidalgo': 194, 'casa': 192, 'estava': 187, 'ainda': 186, 'tempo': 182, 'já': 181, 'mariz': 180, 'project': 176, 'aventureiros': 175, 'momento': 174, 'loredano': 174, 'só': 173, 'mesmo'

In [6]:
vocab_size = 3000
UNK = '<UNK>'
most_frequent_words = [UNK] + [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words)}

vocab_size = len(vocab)
print(vocab_size)
print(most_frequent_words)

3001
['<UNK>', '.', ',', '-', 'a', 'que', 'o', 'de', 'e', 'se', 'um', 'do', 'não', 'uma', 'da', 'os', 'com', 'sua', 'para', 'seu', '!', 'pery', 'as', 'em', 'no', '?', 'por', 'ao', 'como', 'lhe', 'd', 'á', 'tinha', 'era', 'cecilia', 'na', 'é', 'sobre', 'mas', 'elle', 'the', 'dos', 'indio', 'me', 'seus', 'mais', 'antonio', 'quando', 'alvaro', 'disse', 'das', 'vos', 'of', 'ella', 'olhos', 'te', 'senhora', 'menina', 'pela', 'tu', 'depois', 'nos', 'isabel', 'havia', 'gutenberg', 'fidalgo', 'casa', 'estava', 'ainda', 'tempo', 'já', 'mariz', 'project', 'aventureiros', 'momento', 'loredano', 'só', 'mesmo', 'italiano', 'todos', 'pelo', 'vida', 'sem', 'dous', 'to', 'homem', 'eu', 'porque', 'or', 'meio', 'you', 'foi', 'tudo', 'suas', 'ia', 'olhar', 'quem', 'mão', 'aos', 'então', 'meu', 'ou', 'and', 'essa', 'esse', 'rio', 'entre', 'cabeça', 'onde', 'pouco', 'moça', 'tão', 'isto', 'podia', 'in', 'ser', 'minha', 'pai', 'cavalheiro', 'assim', 'moço', 'apenas', 'vez', 'lo', 'esta', 'ha', 'palavra', 's

In [7]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r'\w+|[.,!?-]', sentence.lower())]

def decode_sentence(sentence, most_frequent_words):  # Agradecimento ao Pedro Rodrigues
    return ' '.join([most_frequent_words[c] for c in sentence])

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))
print(decode_sentence(encode_sentence(cleaned_paragraphs[20], vocab), most_frequent_words))

 publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 144, 0, 23, 0, 2, 9, 49, 115, 274, 264, 2667, 13, 1069, 0, 2, 5, 191, 135, 285, 6, 2262, 9, 0, 4, 2670, 1]
<UNK> este <UNK> em <UNK> , se disse ser aquella primeira edição uma prova <UNK> , que algum dia talvez o autor se <UNK> a rever .


In [8]:
# Printing the 20th paragraph, based on the last answer
print(cleaned_paragraphs[20])
print(len(encode_sentence(cleaned_paragraphs[20], vocab)))

# Find keys with a specific value (e.g., 2662)
desired_value = 2670
matching_keys = [key for key, value in vocab.items() if value == desired_value]

# Printing keys with the desired value
print(matching_keys)

 publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
27
['rever']


In [9]:
def input_target(text, context_size):
    context = []
    target = []

    if len(text) - context_size > 0:
        for i in range(len(text) - context_size):
            context.append(text[i:i+context_size])
            target.append([text[i+context_size]])
    
    return context, target

# testando a função
a,b = input_target([11, 2, 11, 0, 2], 3)
for i in range(len(a)):
    print(a[i], '->', b[i])
    
print(a)
print(b)

[11, 2, 11] -> [0]
[2, 11, 0] -> [2]
[[11, 2, 11], [2, 11, 0]]
[[0], [2]]


In [10]:
# divida o dataset em validação/treino com um proporção de 20/80 %. OBS, use random_state=18
train_sentences, val_sentences = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)
print(f'Tamanho de todo o dataset {len(cleaned_paragraphs)}')
print(f'Tamanho do dataset de treino: {len(train_sentences)}')
print(f'Tamanho do dataset de validação: {len(val_sentences)}')

Tamanho de todo o dataset 4892
Tamanho do dataset de treino: 3913
Tamanho do dataset de validação: 979


In [11]:
# Dataset Class with One-hot Encoding
class MyDataset(Dataset):
    def __init__(self, paragraphs, vocab, context_size=9):
        self.paragraphs = paragraphs
        self.vocab = vocab
        self.context_size = context_size
        
        # Pegando o texto, codificando ele baseado no vocab e transformando ele em context e target para o zeroshot
        self.encoded_paragraphs = [encode_sentence(x, self.vocab) for x in self.paragraphs]
        
        context_list = [word for data in self.encoded_paragraphs for word in input_target(data, self.context_size)[0]]
        target_list = [word for data in self.encoded_paragraphs for word in input_target(data, self.context_size)[1]]
     
        self.context_tensor = torch.tensor(context_list)
        self.target_tensor = torch.tensor(target_list)

    def __len__(self):
        return len(self.target_tensor)

    def __getitem__(self, idx):
        return self.context_tensor[idx], self.target_tensor[idx]
    
train_data = MyDataset(train_sentences, vocab, context_size=9)
val_data = MyDataset(val_sentences, vocab, context_size=9)

In [12]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
sample = next(iter(train_loader))

In [13]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import torch

def evaluate_model(model, data_loader, criterion):
    model.eval()
    predictions = []
    true_labels = []
    losses = []
    
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = torch.tensor(inputs, dtype=torch.long)  # Convertendo para Long
            targets = torch.tensor(targets, dtype=torch.float32)
            
            # Removendo a dimensão extra de targets
            targets = targets.squeeze()
            
            output = model(inputs)
            loss = criterion(output.squeeze(), targets)
            losses.append(loss.item())
            
            # Convertendo logits para classes (0 ou 1)
            prediction = (torch.sigmoid(output) > 0.5).float()
            
            predictions.extend(prediction.cpu().numpy())
            true_labels.extend(targets.cpu().numpy())
    
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    avg_loss = np.mean(losses)
    
    return accuracy, precision, recall, f1, avg_loss


In [23]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Função para treinamento e avaliação do modelo
def train_and_evaluate(model, train_loader, val_loader, epochs, criterion, optimizer):
    train_losses = []
    val_losses = []
    train_times = []
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Treinamento
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            inputs = torch.from_numpy(np.array(inputs)).long()  # Convertendo para Long
            targets = torch.from_numpy(np.array(targets)).float()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))
        
        # Avaliação no conjunto de validação
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = torch.from_numpy(np.array(inputs)).long()  # Convertendo para Long
                targets = torch.from_numpy(np.array(targets)).float()
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)
                val_loss += loss.item()
        val_losses.append(val_loss / len(val_loader))
        
        end_time = time.time()
        epoch_time = end_time - start_time
        train_times.append(epoch_time)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Time: {epoch_time:.2f} sec")
    
    return train_losses, val_losses, train_times

class LoRALayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LoRALayer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Adjusted input size
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        # Adjusting dimensions to match fc1 input size
        x = x.view(batch_size, -1)
        
        gate = self.sigmoid(self.fc1(x))
        x = x * gate + (1 - gate) * self.fc2(x)
        return x

class LanguageModelWithLoRA(nn.Module):
    def __init__(self, vocab_size=3000, embedding_dim=128, hidden_size=128):
        super(LanguageModelWithLoRA, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size+1, embedding_dim)
        self.lora = LoRALayer(embedding_dim * 10, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.lora(x)
        x = torch.relu(self.fc2(x))
        return x

# Parâmetros
vocab_size = 3000
embedding_size = 128
hidden_size = 128
epochs = 10
learning_rate = 0.001

# Inicialização do modelo
model = LanguageModelWithLoRA(vocab_size, embedding_size, hidden_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Treinamento do modelo original
print("Treinamento do modelo original:")
train_losses_original, val_losses_original, train_times_original = train_and_evaluate(model, train_loader, val_loader, epochs, criterion, optimizer)

# Ajuste fino com LoRA
print("\nAjuste fino com LoRA:")
train_losses_lora, val_losses_lora, train_times_lora = train_and_evaluate(model, train_loader, val_loader, epochs, criterion, optimizer)

# Avaliação do modelo original
accuracy, precision, recall, f1, avg_loss = evaluate_model(model, val_loader, criterion)
print("\nAvaliação do modelo original:")
print_evaluation(accuracy, precision, recall, f1, avg_loss)

# Avaliação do modelo com LoRA
accuracy_lora, precision_lora, recall_lora, f1_lora, avg_loss_lora = evaluate_model(model, val_loader, criterion)
print("\nAvaliação do modelo com LoRA:")
print_evaluation(accuracy_lora, precision_lora, recall_lora, f1_lora, avg_loss_lora)


Treinamento do modelo original:


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1152 and 1280x128)

In [17]:
# Definição da classe LowRankAdaptation
class LowRankAdaptation(nn.Module):
    def __init__(self, input_size, rank):
        super(LowRankAdaptation, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(input_size, rank))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x):
        return x @ self.weight @ self.weight.T

# Função de treinamento
def train_model_with_lora(model, train_loader, val_loader, epochs, criterion, optimizer, device):
    model.train()
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for batch_data in train_loader:
            inputs, targets = batch_data
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            
            # Ajuste dos pesos das matrizes LoRA
            loss += model.lora_embedding.weight.norm() + model.lora_fc1.weight.norm() + model.lora_fc2.weight.norm()
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        train_loss = running_loss / len(train_loader.dataset)
        
        # Avaliação no conjunto de validação
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for val_batch_data in val_loader:
                val_inputs, val_targets = val_batch_data
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_targets.squeeze())
                
                val_running_loss += val_loss.item() * val_inputs.size(0)
            
            val_loss = val_running_loss / len(val_loader.dataset)
            
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Criando instância do modelo com LoRA
model_with_lora = LanguageModelWithLoRA(vocab_size)

# Definindo critério e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_with_lora.parameters(), lr=0.08)

# Movendo o modelo e o critério para o dispositivo
model_with_lora.to(device)
criterion.to(device)

# Treinamento do modelo com LoRA
train_model_with_lora(model_with_lora, train_loader, val_loader, epochs=10, criterion=criterion, optimizer=optimizer, device=device)


Epoch [1/10], Train Loss: 8.2688, Val Loss: 8.0074
Epoch [2/10], Train Loss: 8.1279, Val Loss: 8.0074
Epoch [3/10], Train Loss: 8.1279, Val Loss: 8.0073
Epoch [4/10], Train Loss: 8.1279, Val Loss: 8.0073
Epoch [5/10], Train Loss: 8.1278, Val Loss: 8.0073
Epoch [6/10], Train Loss: 8.1278, Val Loss: 8.0073
Epoch [7/10], Train Loss: 8.1278, Val Loss: 8.0073
Epoch [8/10], Train Loss: 8.1278, Val Loss: 8.0073
Epoch [9/10], Train Loss: 8.1278, Val Loss: 8.0073
Epoch [10/10], Train Loss: 8.1278, Val Loss: 8.0073
